<a href="https://colab.research.google.com/github/peterchang0414/lecun1989-flax/blob/main/lecun_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1989 Reproduction

In [ ]:
!pip install -q flax

     |████████████████████████████████| 184 kB 5.4 MB/s 
     |████████████████████████████████| 136 kB 49.9 MB/s 
     |████████████████████████████████| 70 kB 6.6 MB/s 


In [ ]:
import jax
import jax.numpy as jnp

from flax import linen as nn
from torchvision import datasets

# Adapted from https://github.com/karpathy/lecun1989-repro/blob/master/prepro.py
def get_datasets(n_tr, n_te):
  train_test = {}
  for split in {'train', 'test'}:
    data = datasets.MNIST('./data', train=split=='train', download=True)
    
    n = n_tr if split == 'train' else n_te
    key = jax.random.PRNGKey(42)
    rp = jax.random.permutation(key, len(data))[:n]

    X = jnp.full((n, 16, 16, 1), 0.0, dtype=jnp.float32)
    Y = jnp.full((n, 10), -1.0, dtype=jnp.float32)
    for i, ix in enumerate(rp):
      I, yint = data[int(ix)]
      xi = jnp.array(I, dtype=np.float32) / 127.5 - 1.0
      xi = jax.image.resize(xi, (16, 16), 'bilinear')
      X = X.at[i].set(np.expand_dims(xi, axis=2))
      Y = Y.at[i, yint].set(1.0)
    train_test[split] = (X, Y)
  return train_test

In [ ]:
from flax import linen as nn
from flax.training import train_state
from flax.linen.activation import tanh
import optax
from typing import Callable

class Net(nn.Module):
  bias_init: Callable = nn.initializers.zeros
  kernel_init: Callable = nn.initializers.uniform()

  @nn.compact
  def __call__(self, x):
    # For weight initialization, Karpathy used numerator of 2.4 
    # which is very close to sqrt(6) = 2.449... used by he_uniform()
    # By default, weight-sharing forces bias-sharing and therefore
    # we add the bias separately.
    x = jnp.pad(x, [(0,0),(2,2),(2,2),(0,0)], constant_values=-1.0)
    x = nn.Conv(features=12, kernel_size=(5,5), strides=2, padding='VALID',
                use_bias=False, kernel_init=self.kernel_init)(x)
    bias1 = self.param('bias1', self.bias_init, (8, 8, 12))
    x = tanh(x + bias1)
    x = jnp.pad(x, [(0,0),(2,2),(2,2),(0,0)], constant_values=-1.0)
    x1, x2, x3 = (x[..., 0:8], x[..., 4:12], 
                  jnp.concatenate((x[..., 0:4], x[..., 8:12]), axis=-1))
    slice1 = nn.Conv(features=4, kernel_size=(5,5), strides=2, padding='VALID', 
                     use_bias=False, kernel_init=self.kernel_init)(x1)
    slice2 = nn.Conv(features=4, kernel_size=(5,5), strides=2, padding='VALID',
                     use_bias=False, kernel_init=self.kernel_init)(x2)
    slice3 = nn.Conv(features=4, kernel_size=(5,5), strides=2, padding='VALID',
                     use_bias=False, kernel_init=self.kernel_init)(x3)
    x = jnp.concatenate((slice1, slice2, slice3), axis=-1)
    bias2 = self.param('bias2', self.bias_init, (4, 4, 12))
    x = tanh(x + bias2)
    x = x.reshape((x.shape[0], -1))
    x = nn.Dense(features=30, use_bias=False)(x)
    bias3 = self.param('bias3', self.bias_init, (30,))
    x = tanh(x + bias3)
    x = nn.Dense(features=10, use_bias=False)(x)
    bias4 = self.param('bias4', nn.initializers.constant(-1.0), (10,))
    x = tanh(x + bias4)

    return x

In [ ]:
@jax.jit
def eval_step(params, X, Y):
  Yhat = Net().apply({'params': params}, X)
  loss = jnp.mean((Yhat - Y)**2)
  err = jnp.mean(jnp.argmax(Y, -1) != jnp.argmax(Yhat, -1)).astype(float)
  return loss, err

In [ ]:
def eval_split(data, split, params):
  X, Y = data[split]
  loss, err = eval_step(params, X, Y)
  print(f"eval: split {split:5s}. loss {loss:e}. error {err*100:.2f}%. misses: {int(err*Y.shape[0])}")

In [ ]:
from jax import value_and_grad
import optax
from flax.training import train_state

def create_train_state(key, lr, X):
  model = Net()
  params = model.init(key, X)['params']
  sgd_opt = optax.sgd(lr)
  return train_state.TrainState.create(apply_fn=model.apply, params=params, tx=sgd_opt)

@jax.jit
def train_step(state, X, Y):
  def loss_fn(params):
    Yhat = Net().apply({'params': params}, X)
    loss = jnp.mean((Yhat - Y)**2)
    err = jnp.mean(jnp.argmax(Y, -1) != jnp.argmax(Yhat, -1)).astype(float)
    return loss, err
  (_, Yhats), grads = jax.value_and_grad(loss_fn, has_aux=True)(state.params)
  state = state.apply_gradients(grads=grads)
  return state

def train_one_epoch(state, X, Y):
  for step_num in range(X.shape[0]):
    x, y = jnp.expand_dims(X[step_num], 0), jnp.expand_dims(Y[step_num], 0)
    state = train_step(state, x, y)
  return state

def train(key, data, epochs, lr):
  Xtr, Ytr = data['train']
  Xte, Yte = data['test']
  train_state = create_train_state(key, lr, Xtr)
  for epoch in range(epochs):
    print(f"epoch {epoch+1}")
    train_state = train_one_epoch(train_state, Xtr, Ytr)
    for split in ['train', 'test']:
      eval_split(data, split, train_state.params)


In [ ]:
data = get_datasets(7291, 2007)

In [ ]:
key, _ = jax.random.split(jax.random.PRNGKey(42))

train(key, data, 23, 0.03)

KeyboardInterrupt: ignored

Results:

```
epoch 23
eval: split train. loss 6.397844e-03. error 0.97%. misses: 71
eval: split test . loss 2.608742e-02. error 3.84%. misses: 77
```



# "Modern" Adjustments

In [1]:
!pip install -q flax

     |████████████████████████████████| 184 kB 5.1 MB/s 
     |████████████████████████████████| 136 kB 64.0 MB/s 
     |████████████████████████████████| 70 kB 8.7 MB/s 


In [2]:
import jax
import jax.numpy as jnp

from flax import linen as nn
from torchvision import datasets

# Adapted from https://github.com/karpathy/lecun1989-repro/blob/master/prepro.py
def get_datasets(n_tr, n_te):
  train_test = {}
  for split in {'train', 'test'}:
    data = datasets.MNIST('./data', train=split=='train', download=True)
    
    n = n_tr if split == 'train' else n_te
    key = jax.random.PRNGKey(42)
    rp = jax.random.permutation(key, len(data))[:n]

    X = jnp.full((n, 16, 16, 1), 0.0, dtype=jnp.float32)
    Y = jnp.full((n, 10), 0, dtype=jnp.float32)
    for i, ix in enumerate(rp):
      I, yint = data[int(ix)]
      xi = jnp.array(I, dtype=np.float32) / 127.5 - 1.0
      xi = jax.image.resize(xi, (16, 16), 'bilinear')
      X = X.at[i].set(np.expand_dims(xi, axis=2))
      Y = Y.at[i, yint].set(1.0)
    train_test[split] = (X, Y)
  return train_test

In [4]:
from flax import linen as nn
from flax.training import train_state
from flax.linen.activation import tanh
import optax
from typing import Callable

class Net(nn.Module):
  training: bool
  bias_init: Callable = nn.initializers.zeros
  kernel_init: Callable = nn.initializers.uniform()

  @nn.compact
  def __call__(self, x):
    # For weight initialization, Karpathy used numerator of 2.4 
    # which is very close to sqrt(6) = 2.449... used by he_uniform()
    # By default, weight-sharing forces bias-sharing and therefore
    # we add the bias separately.
    if self.training:
      augment_rng = self.make_rng('aug')
      # self.sow('intermediates', 'aug_rng', augment_rng)
      shift_x, shift_y = jax.random.randint(augment_rng, (2,), -1, 2)
      x = jnp.roll(x, (shift_x, shift_y), (1, 2))

    x = jnp.pad(x, [(0,0),(2,2),(2,2),(0,0)], constant_values=-1.0)
    x = nn.Conv(features=12, kernel_size=(5,5), strides=2, padding='VALID',
                use_bias=False, kernel_init=self.kernel_init)(x)
    bias1 = self.param('bias1', self.bias_init, (8, 8, 12))
    x = tanh(x + bias1)
    x = jnp.pad(x, [(0,0),(2,2),(2,2),(0,0)], constant_values=-1.0)
    x1, x2, x3 = (x[..., 0:8], x[..., 4:12], 
                  jnp.concatenate((x[..., 0:4], x[..., 8:12]), axis=-1))
    slice1 = nn.Conv(features=4, kernel_size=(5,5), strides=2, padding='VALID', 
                     use_bias=False, kernel_init=self.kernel_init)(x1)
    slice2 = nn.Conv(features=4, kernel_size=(5,5), strides=2, padding='VALID',
                     use_bias=False, kernel_init=self.kernel_init)(x2)
    slice3 = nn.Conv(features=4, kernel_size=(5,5), strides=2, padding='VALID',
                     use_bias=False, kernel_init=self.kernel_init)(x3)
    x = jnp.concatenate((slice1, slice2, slice3), axis=-1)
    bias2 = self.param('bias2', self.bias_init, (4, 4, 12))
    x = tanh(x + bias2)
    x = x.reshape((x.shape[0], -1))
    x = nn.Dense(features=30, use_bias=False)(x)
    bias3 = self.param('bias3', self.bias_init, (30,))
    x = tanh(x + bias3)
    x = nn.Dense(features=10, use_bias=False)(x)
    bias4 = self.param('bias4', nn.initializers.constant(-1.0), (10,))
    x = x + bias4

    return x

In [17]:
from jax import value_and_grad
import optax
from flax.training import train_state

def learning_rate_fn(initial_rate, epochs, steps_per_epoch):
  return optax.linear_schedule(init_value=initial_rate, end_value=initial_rate/3,
                               transition_steps=epochs*steps_per_epoch)

def create_train_state(key, X, lr_fn):
  model = Net(training=True)
  key1, key2 = jax.random.split(key)
  params = model.init({'params': key1, 'aug': key2}, X)['params']
  opt = optax.adamw(lr_fn)
  return train_state.TrainState.create(apply_fn=model.apply, params=params, tx=opt)

@jax.jit
def train_step(state, X, Y, aug_rng=jax.random.PRNGKey(0),
               dropout_rng=jax.random.PRNGKey(0)):
  aug_rng = jax.random.fold_in(aug_rng, state.step)
  dropout_rng, _ = jax.random.split(aug_rng)
  def loss_fn(params):
    Yhat = Net(training=True).apply({'params': params}, X, 
                                           rngs={'aug': aug_rng})
    loss = jnp.mean(optax.softmax_cross_entropy(logits=Yhat, labels=Y))
    err = jnp.mean(jnp.argmax(Y, -1) != jnp.argmax(Yhat, -1)).astype(float)
    return loss, err
  (_, Yhats), grads = jax.value_and_grad(loss_fn, has_aux=True)(state.params)
  state = state.apply_gradients(grads=grads)
  return state

def train_one_epoch(state, X, Y):
  for step_num in range(X.shape[0]):
    x, y = jnp.expand_dims(X[step_num], 0), jnp.expand_dims(Y[step_num], 0)
    state = train_step(state, x, y)
  return state

def train(key, data, epochs, lr):
  Xtr, Ytr = data['train']
  Xte, Yte = data['test']
  lr_fn = learning_rate_fn(lr, epochs, Xtr.shape[0])
  train_state = create_train_state(key, Xtr, lr_fn)
  for epoch in range(epochs):
    print(f"epoch {epoch+1} with learning rate {lr_fn(train_state.step):.6f}")
    train_state = train_one_epoch(train_state, Xtr, Ytr)
    for split in ['train', 'test']:
      eval_split(data, split, train_state.params)


In [6]:
@jax.jit
def eval_step(params, X, Y):
  Yhat = Net(training=False).apply({'params': params}, X)
  loss = jnp.mean(optax.softmax_cross_entropy(logits=Yhat, labels=Y))
  err = jnp.mean(jnp.argmax(Y, -1) != jnp.argmax(Yhat, -1)).astype(float)
  return loss, err

def eval_split(data, split, params):
  X, Y = data[split]
  loss, err = eval_step(params, X, Y)
  print(f"eval: split {split:5s}. loss {loss:e}. error {err*100:.2f}%. misses: {int(err*Y.shape[0])}")

In [11]:
data = get_datasets(7291, 2007)

In [18]:
key, _ = jax.random.split(jax.random.PRNGKey(42))

train(key, data, 60, 3e-4)

epoch 1 with learning rate 0.000300
eval: split train. loss 5.425038e-01. error 13.44%. misses: 980
eval: split test . loss 5.180030e-01. error 12.71%. misses: 255
epoch 2 with learning rate 0.000297
eval: split train. loss 3.860081e-01. error 10.52%. misses: 767
eval: split test . loss 3.717132e-01. error 10.31%. misses: 207
epoch 3 with learning rate 0.000293
eval: split train. loss 3.163782e-01. error 9.22%. misses: 672
eval: split test . loss 3.055812e-01. error 9.47%. misses: 190
epoch 4 with learning rate 0.000290
eval: split train. loss 2.549224e-01. error 7.43%. misses: 542
eval: split test . loss 2.393445e-01. error 6.38%. misses: 128
epoch 5 with learning rate 0.000287
eval: split train. loss 2.062259e-01. error 6.13%. misses: 447
eval: split test . loss 1.851927e-01. error 5.28%. misses: 106
epoch 6 with learning rate 0.000283
eval: split train. loss 1.817566e-01. error 5.35%. misses: 390
eval: split test . loss 1.699917e-01. error 5.28%. misses: 106
epoch 7 with learning ra

Change 1: replace tanh on last layer with FC and use softmax. Lower learning rate to 0.01

```
epoch 23
eval: split train. loss 7.162272e-03. error 0.05%. misses: 4
eval: split test . loss 1.687743e-01. error 4.14%. misses: 83

```

Change 2: change from SGD to AdamW with LR 3e-4, double epochs to 46, decay LR to 1e-4 over the course of training.

```
epoch 46
eval: split train. loss 1.890260e-03. error 0.04%. misses: 2
eval: split test . loss 1.953933e-01. error 4.04%. misses: 81
```

Change 3: Introduce data augmentation, e.g. a shift by at most 1 pixel in both x/y directions, and bump up training time to 60 epochs.

```
epoch 60
eval: split train. loss 5.098452e-02. error 1.65%. misses: 120
eval: split test . loss 9.166716e-02. error 2.59%. misses: 52
```

Change 4: add dropout at layer H3, shift activation function to relu, and bring up iterations to 80.

```
# 코드로 형식 지정됨
```




